# Imports

In [2]:
# Packages
import pandas as pd

# Helper functions
from ipynb.fs.full.functions import *

# Other notebooks for combining code
from ipynb.fs.full.semrush_call import execute_call
from ipynb.fs.full.text_division_and_styling import LoopGetHeaderCountFromURL, LoopGetListsCountFromURL
from ipynb.fs.full.semrush_beoordeling_concurrentieniveau import getKeywordDifficulty 
from ipynb.fs.full.InboundOutboundScraper import loop_get_all_website_links
from ipynb.fs.full.SemRush_InboundOutbound_Backlink import get_all_website_links_semrush
from ipynb.fs.full.img_scraper import *
from ipynb.fs.full.content_recency import GetContentRecency, days_between_today_and_last_modified
from ipynb.fs.full.keyword_density import GetKeywordDensity
from ipynb.fs.full.KeywordInText import getKeywordmain_dataInText
from ipynb.fs.full.website_speed import GetWebsiteSpeed 

# Executing of functions
## Variables for keyword search

In [3]:
keyword = 'nieuws' # What keyword do you want to call for?
result_count = '99' # How many keyword results do you want to gather data for? Maximum for our license is 99

# Do you want to overwrite the SEMrush keyword call if it has already been executed before?
# This is to save on SEMrush token usage
potential_overwrite = False

# Do you want to save the result of the end dataframe?
save = True

## Creating dataframes

In [4]:
def ExecuteAllCalls():
    # Define variables for the SEMrush call and gather keyword data
    export_columns = 'Dn,Ur'
    endpoint_url = 'https://api.semrush.com/'
    main_dataframe = execute_call(keyword, result_count, export_columns, endpoint_url, potential_overwrite)
    main_dataframe
    
    # Add used keyword to the dataframe
    keyword_list = []
    keyword_list += int(result_count) * [keyword]
    main_dataframe['Keyword'] = keyword_list
    
    # Executing functions from other notebooks to gather data
    print("  0% - Executing getKeywordDifficulty")
    KeywordDifficultyFunction = getKeywordDifficulty(keyword, potential_overwrite)
    KeywordDifficultyList = []
    KeywordDifficultyList += int(result_count) * [KeywordDifficultyFunction['Kd'][0]]
    main_dataframe['Kd'] = KeywordDifficultyList
    print(" 10% - Executing GetKeywordDensity")
    main_dataframe = main_dataframe.merge(GetKeywordDensity(keyword, main_dataframe), how='left', left_on='Ur', right_on='URL')
    print(" 20% - Executing LoopGetHeaderCountFromURL")
    main_dataframe['HeaderCount'] = LoopGetHeaderCountFromURL(main_dataframe)
    print(" 30% - Executing LoopGetListsCountFromURL")
    main_dataframe['ListsCount'] = LoopGetListsCountFromURL(main_dataframe)
    print(" 40% - Executing loop_get_all_website_links")
    website_links = loop_get_all_website_links(main_dataframe)
    main_dataframe['InternalLinksSoup'] = website_links[0]
    main_dataframe['ExternalLinksSoup'] = website_links[1]
    print(" 50% - Executing get_all_website_links_semrush")
    main_dataframe['InternalLinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'internal', potential_overwrite)
    main_dataframe['ExternalLinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'external', potential_overwrite)
    main_dataframe['BacklinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'backlink', potential_overwrite)
    main_dataframe['PageAScore'] = get_all_website_links_semrush(main_dataframe, 'page_ascore', potential_overwrite)
    main_dataframe['DomainAScore'] = get_all_website_links_semrush(main_dataframe, 'domain_ascore', potential_overwrite)
    main_dataframe['AScore'] = get_all_website_links_semrush(main_dataframe, 'ascore', potential_overwrite)
    print(" 60% - Executing createDF") 
    main_dataframe = main_dataframe.merge(createDF(main_dataframe).drop('Domain', axis=1), how='left', left_on='Ur', right_on='url')
    print(" 70% - Executing getKeywordmain_dataInText")
    main_dataframe = main_dataframe.rename(columns={'META-Title': 'METATitle', 'META-Description': 'METADescription', 'Image Tags': 'ImageTags'}) # Rename columns for bugfix
    main_dataframe = getKeywordmain_dataInText(main_dataframe)
    print(" 80% - Executing GetContentRecency and days_between_today_and_last_modified")
    main_dataframe = main_dataframe.merge(GetContentRecency(main_dataframe), how='left', left_on='Ur', right_on='URL')
    main_dataframe = days_between_today_and_last_modified(main_dataframe)
    print(" 90% - Executing GetWebsiteSpeed")
    main_dataframe = main_dataframe.merge(GetWebsiteSpeed(keyword, main_dataframe), how='left', left_on='Ur', right_on='URL')
    
    # Set correct index, potentially save the output and return the dataframe
    main_dataframe = main_dataframe.set_index('Number').drop(['url', 'URL', 'URL_x', 'URL_y'], axis=1) # Drop URL because it stays in there from the merge
    main_dataframe_hash = hash_variable(main_dataframe)
    if save == True:
        main_dataframe.to_csv(f"calls\dataframe-{main_dataframe_hash}.csv")
    print("100% - Done")
    return main_dataframe

In [5]:
main_dataframe = ExecuteAllCalls()

Deze call is al een keer uitgevoerd, we gaan verder met de bestaande resultaten.
Dit bestand is laatst gewijzigd op: Fri May 21 14:59:44 2021
  0% - Executing getKeywordDifficulty
Deze call is al een keer uitgevoerd, we gaan verder met de bestaande resultaten.
Dit bestand is laatst gewijzigd op: Fri May 21 14:59:44 2021
 10% - Executing GetKeywordDensity
 20% - Executing LoopGetHeaderCountFromURL
 30% - Executing LoopGetListsCountFromURL
 40% - Executing loop_get_all_website_links
 50% - Executing get_all_website_links_semrush
 60% - Executing createDF
 70% - Executing getKeywordmain_dataInText


C:\Users\wafoe\OneDrive - Windesheim Office365\School\6 ICTDSe\Proof of Concept\SEO-De-Nieuwe-Zaak\KeywordInText.ipynb:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "metadata": {},
C:\Users\wafoe\OneDrive - Windesheim Office365\School\6 ICTDSe\Proof of Concept\SEO-De-Nieuwe-Zaak\KeywordInText.ipynb:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "    df['MetaDescriptionExists'] = 0\n",
C:\Users\wafoe\OneDrive - Windesheim Office365\School\6 ICTDSe\Proof of Concept\SEO-De-Nieuwe-Zaak\KeywordInText.ipynb:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

 80% - Executing GetContentRecency
 90% - Executing GetWebsiteSpeed
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
one failed url added
100% - Done


In [7]:
main_dataframe.head(99)

,Dn,Ur,Keyword,Kd,keyword_density_percentage,HeaderCount,ListsCount,InternalLinksSoup,ExternalLinksSoup,InternalLinksSEMrush,...,KeyWordInMetaDescription,KeyWordInUrl,KeyWordInImageTags,original_publication,last_modified,days_between,FCP,LCP,FID,CLS
Number,,,,,,,,,,,,,,,,,,,,,
0,nu.nl,https://www.nu.nl/algemeen,nieuws,71,0.08,4,25,256.0,107.0,7.0,...,1,0,0,2021-05-24,2021-05-24,0,NaN,NaN,NaN,NaN
1,nos.nl,https://nos.nl/nieuws,nieuws,71,1.10,3,6,67.0,6.0,4470.0,...,0,1,0,2021-05-24,2021-05-24,0,NaN,NaN,NaN,NaN
2,telegraaf.nl,https://www.telegraaf.nl/nieuws,nieuws,71,0.06,2,16,194.0,43.0,43.0,...,0,1,0,2021-05-24,2021-05-24,0,NaN,NaN,NaN,NaN
3,ad.nl,https://www.ad.nl/nieuws/,nieuws,71,0.00,0,0,0.0,0.0,18.0,...,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nieuws.nl,https://nieuws.nl/,nieuws,71,0.51,3,15,107.0,11.0,11.0,...,0,1,0,2014-11-05,2014-11-05,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,bndestem.nl,https://www.bndestem.nl/,nieuws,71,0.00,0,0,0.0,0.0,169.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,greenpeace.org,https://www.greenpeace.org/nl/nieuws/,nieuws,71,0.65,4,7,90.0,8.0,NaN,...,0,1,0,2020-04-21,2020-04-21,0,NaN,NaN,NaN,NaN
96,rd.nl,https://www.rd.nl/laatste-nieuws?page=1&catego...,nieuws,71,0.32,3,20,75.0,28.0,NaN,...,0,1,0,2021-05-24,2021-05-24,0,NaN,NaN,NaN,NaN


In [ ]:
# main_dataframe3 = main_dataframe2.replace('None', 'NaN')
# main_dataframe3.transpose()
# main_dataframe3.to_csv("calls\dataframe_voor_analyse2.csv")